In [1]:
# import libraries
import pandas as pd
import random
import os

In [2]:
%%time
# import raw training data
root_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_path = '/data/'
filename = 'unsampled_input_data.csv'
df = pd.read_csv(root_path + data_path + filename)

CPU times: user 4min 12s, sys: 1min 44s, total: 5min 56s
Wall time: 6min 15s


In [3]:
# sample the session ids
# doesn't work when too many indexes...
# def some(x, n):
#     return x.loc[random.sample(x.index.values.tolist(), n),:]

In [10]:
# sample the session ids
def some(x, n):
    return x.ix[random.sample(x.index.values.tolist(), n)]

In [4]:
%%time
# remove unwanted columns
df.drop(['campaign_id', 'geoip_region', 'geoip_city_location_id'], axis=1, inplace=True)

CPU times: user 9.49 s, sys: 49 s, total: 58.5 s
Wall time: 1min 14s


In [5]:
%%time
# create checkout feature on page level
df['checkout_page'] = df['is_checkout_page'].str.contains(u't').astype(int)
# remove original checkout column
df.drop(['is_checkout_page'], axis=1, inplace=True)

# create checkout feature on session level
df_temp_co = df.groupby(by=['session_id'])['checkout_page'].max()
dfj_co = pd.DataFrame()
dfj_co['session_id'] = df_temp_co.index
dfj_co['checkout_visit'] = df_temp_co.values
dfj_co.columns = ['session_id','checkout_visit']
df = pd.merge(df, dfj_co, how='left', on='session_id')

# remove original checkout_page column
df.drop(['checkout_page'], axis=1, inplace=True)

CPU times: user 1min 47s, sys: 2min 47s, total: 4min 34s
Wall time: 5min 11s


# Check the primaric traffic sources

In [ ]:
# for info
len(df.query("checkout_visit == 1").groupby(['session_id','source_first']))
df.query("checkout_visit == 1").groupby(['source_first']).count().sort_values(by='session_id', ascending=False)

# Rebalance dataset if imbalanced

In [8]:
%%time
# list all unique values of session id where checkout page = 1
converters = pd.Series(df.query("checkout_visit > 0").session_id.unique()) 
# list all unique values of session id where checkout page = 0
non_converters = pd.Series(df.query("checkout_visit == 0").session_id.unique())

In [11]:
%%time
# filter the original data frame to only retain the selected session ids
ind = some(non_converters,len(converters))
df_filtered = df[df.session_id.isin(ind.values) | df.query("checkout_visit > 0").session_id]

/Users/bbelmudez/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


CPU times: user 38 s, sys: 37.4 s, total: 1min 15s
Wall time: 1min 26s


In [12]:
# # Verify if sampling is successful, i.e. both classes are equal size 
# # list all unique values of session id where checkout page = 1
# cnvrtrs_unique = len(pd.Series(df_filtered.query("checkout_visit > 0").session_id.unique()))
# # list all unique values of session id where checkout page = 0
# nn_cnvrtrs_unique = len(pd.Series(df_filtered.query("checkout_visit == 0").session_id.unique()))
# # assert class sizes
# if cnvrtrs_unique != nn_cnvrtrs_unique:
#     print("Resampling failed!")
# else:
#     print("Resampling succeeded")

## Create features ##

In [14]:
del df
df = df_filtered

In [15]:
%%time
# create traffic source feature on session level
df_temp_so = df.groupby(df.session_id)['pageview_time'].min()
df_so = df.loc[df['session_id'].isin(df_temp_so.index) &
               df['pageview_time'].isin(df_temp_so.values), ['session_id', 'source']]
df_so.reset_index(inplace=True, drop=True)
df_so.columns = ['session_id', 'source_first']
df = pd.merge(df, df_so, how='left', on='session_id')

# alternatively, category encode it
df['source_id'] = pd.Categorical(df.source_first).codes

# remove original source column
df.drop(['source'], axis=1, inplace=True)
df.drop(['source_first'], axis=1, inplace=True)

CPU times: user 11 s, sys: 363 ms, total: 11.3 s
Wall time: 11.5 s


In [16]:
# %%time
# format the date column into pandas datetime format
df['date_fmt'] = pd.to_datetime(df['pageview_time'], infer_datetime_format=True, errors='coerce')

# remove original date column
df.drop(['pageview_time'], axis=1, inplace=True)

In [17]:
# %%time
# inferring device type from user_agent
device_list = []
for row in df.index:
    try:
        device = df['user_agent'][row].split('0 (')[1].split(') ')[0].split(';')[0].split(' ')[0]
        device_list.append(device)
    except IndexError:
        device_list.append(np.nan)
    except AttributeError:
        device_list.append(np.nan)

df['device'] = Series(device_list, name = 'device')

other_device = set(unique(device_list)) - {'Windows','Macintosh','iPad','iPhone','Android',
                                           'iPod','Linux','Mobile','compatible','BB10'}
df.device = df.device.replace('iPad','tablet')
df.device = df.device.replace('Linux','phone')
df.device = df.device.replace('Android','phone')
df.device = df.device.replace('compatible','desktop')
df.device = df.device.replace('Mobile','mobile')
df.device = df.device.replace('BB10','mobile')
df.device = df.device.replace('Macintosh','desktop')
df.device = df.device.replace('iPhone','mobile')
df.device = df.device.replace('iPod','mobile')
df.device = df.device.replace('Windows','desktop')

for name in other_device:
    df.device = df.device.replace(name,'other')

# label encode the device dimension    
# le_dev = LabelEncoder()
# le_dev.fit(df['device'].unique())
# df['device_id'] = le_dev.transform(df.device)

# alternatively, category encode it
df['device_id'] = pd.Categorical(df.device).codes

# remove original user agent column
df.drop(['user_agent'], axis=1, inplace=True)
df.drop(['device'], axis=1, inplace=True)

In [18]:
# %%time
# label encode geo country codes
# le_cc = LabelEncoder()
# le_cc.fit(df['geoip_country_code'].unique())
# df['country_id'] = le_cc.transform(df.geoip_country_code)

# alternatively, category encode geo country codes
df['country_id'] = pd.Categorical(df.geoip_country_code).codes

# clean up city names and label encode
df.geoip_city = df.geoip_city.str.replace('Ĺ\x99', 'ø', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('Äš', 'å', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('Ä\x87', 'æ', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('Ĺ\x98', 'ø', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('Äş', 'å', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('Ăś', 'ø', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('Ă¤', 'ø', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('ĂŠ', 'é', n=-1, case=True, flags=0)
df.geoip_city = df.geoip_city.str.replace('ĂĄ', 'á', n=-1, case=True, flags=0)
replacers = [None, np.nan, "None", "NaN", "nan"]
df.geoip_city.replace(replacers, '', inplace=True)
df.geoip_city = df.geoip_city.str.lower()

# label encode geo country codes
# le_cn = LabelEncoder()
# le_cn.fit(df['geoip_city'].unique())
# df['city_id'] = le_cn.transform(df.geoip_city)

# alternatively, category encode it
df['city_id'] = pd.Categorical(df.geoip_city).codes

# remove original country column
df.drop(['geoip_country_code'], axis=1, inplace=True)
df.drop(['geoip_city'], axis=1, inplace=True)

In [19]:
# %%time
# calculate visit duration
for sess_id in df.session_id.unique():
    tmax = df.loc[df.session_id == sess_id, 'date_fmt'].max()
    tmin = df.loc[df.session_id == sess_id, 'date_fmt'].min()
    df.loc[df.session_id == sess_id, 'visit_len'] = pd.Timedelta(tmax - tmin).seconds

In [20]:
# calculate number of pageviews per visit
df_temp_pv = df.groupby(by='session_id')['pageview_url'].count()
dfj_pv = pd.DataFrame()
dfj_pv['session_id'] = df_temp_pv.index
dfj_pv['pageviews'] = df_temp_pv.values
dfj_pv.columns = ['session_id','pageviews']
df = pd.merge(df, dfj_pv, how='left', on='session_id')

In [21]:
# has visited customer service page
df['cust_serv_page'] = df['pageview_url'].str.contains(u'kundeservice').astype(int)

# attribute the flag to the session
df_temp_cs = df.groupby(by='session_id')['cust_serv_page'].max()
dfj_cs = pd.DataFrame()
dfj_cs['session_id'] = df_temp_cs.index
dfj_cs['cust_serv_visit'] = df_temp_cs.values
dfj_cs.columns = ['session_id','cust_serv_visit']
df = pd.merge(df, dfj_cs, how='left', on='session_id')

# remove page-level variable
df.drop(['cust_serv_page'], axis=1, inplace=True)

In [22]:
# calculate starting timestamp for a session
df_temp_psess = df[['session_id','date_fmt']]
df_temp_psess.is_copy = False

# aggregate the column by session id and take the min date, i.e. when the session started
dfg_psess = df_temp_psess.groupby(by='session_id')['date_fmt'].min()
dfj_psess = pd.DataFrame()
dfj_psess['session_id'] = dfg_psess.index
dfj_psess['date'] = dfg_psess.values
dfj_psess.columns = ['session_id','date']
df = pd.merge(df, dfj_psess, how='left', on='session_id')

# remove page-level date
df.drop(['date_fmt'], axis=1, inplace=True)

In [23]:
# remove URL info - i.e. groupby session_id
df.drop(['pageview_url'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [24]:
# %%time
# calculate nb of past visits 

# add a count column for 
df['count'] = 1

# sort the df by session_id
df.sort_values(['visitor_id','session_id'], ascending=True, inplace=True)

# create a column with cumulated values
for user_id in df.visitor_id.unique():
    df.loc[df.visitor_id == user_id, 'nb_past_visits'] = df.loc[df.visitor_id == user_id, 'count']\
                                                           .cumsum() - 1
    df.loc[df.visitor_id == user_id, 'is_customer'] = df.loc[df.visitor_id == user_id, 'cust_serv_visit']\
                                                        .cumsum() - 1
        
df.is_customer.replace(to_replace=-1, value=0, inplace=True)# drop date_min and count
df.drop(['count'], axis=1, inplace=True)

In [25]:
# %%time
# calculate the number of past conversions

# sort the df by session_id
df.sort_values(['visitor_id','session_id'], ascending=True, inplace=True)

# create a column with cumulated values
for user_id in df.visitor_id.unique():
    df_count = df.loc[df.visitor_id == user_id, 'checkout_visit'].shift()
    df_count.fillna(0, inplace=True)
    df.loc[df.visitor_id == user_id, 'nb_past_checkouts'] = df_count.cumsum()

CPU times: user 5min 10s, sys: 1.56 s, total: 5min 11s
Wall time: 5min 12s


In [26]:
# %%time
# calculate the last number of pageviews and session duration

# sort the df by session_id
df.sort_values(['visitor_id','session_id'], ascending=True, inplace=True)

# create a column with cumulated values
for user_id in df.visitor_id.unique():
    df_pv_count = df.loc[df.visitor_id == user_id, 'pageviews'].shift(1)
    df_sd_count = df.loc[df.visitor_id == user_id, 'visit_len'].shift(1)
    df_pv_count.fillna(0, inplace=True)
    df_sd_count.fillna(0, inplace=True)
    df.loc[df.visitor_id == user_id, 'last_pageviews'] = df_pv_count
    df.loc[df.visitor_id == user_id, 'last_visit_len'] = df_sd_count

CPU times: user 10min 19s, sys: 4.11 s, total: 10min 23s
Wall time: 10min 25s


## Save feature dataframe to file

In [28]:
# save the dataframe in a file
df.to_pickle(root_path + data_path + 'input_file.pkl')